In [1]:
'''
Integrated Quadcopter Simulation with VLM Aerodynamic Model
This code integrates a Vortex Lattice Method (VLM) aerodynamic model 
with a quadcopter dynamics and control framework.
'''

import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import copy
import pyvista as pv

# Import custom VLM modules
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from vlm import VLM
from wind import WindField

class QuadcopterVLM:
    def __init__(self, pos, vel, angle, ang_vel, pos_ref, dt, vlm_solver, quad_propeller_mesh):
        # Initialize parameters 
        mass = 0.68  # kg
        gravity = 9.8  # acceleration due to gravity, m/s^2
        num_motors = 4  # number of motors on the vehicle
        
        # Initial state variables
        self.pos = np.array(pos, dtype=float)  # position in inertial frame [x,y,z] (m)
        self.vel = np.array(vel, dtype=float)  # velocity in inertial frame [x_dot, y_dot, z_dot] (m/s)
        self.angle = np.array(angle, dtype=float)  # orientation in radians [roll, pitch, yaw]
        self.ang_vel = np.array(ang_vel, dtype=float)  # angular velocity (rad/s)
        self.lin_acc = np.zeros(3, dtype=float)  # linear acceleration in inertial frame
        self.ang_acc = np.zeros(3, dtype=float)  # angular acceleration in inertial frame

        # Desired reference states
        self.pos_ref = np.array(pos_ref, dtype=float)  # desired position [x, y, z] (m)
        self.vel_ref = np.zeros(3, dtype=float)  # desired velocity (m/s)
        self.lin_acc_ref = np.zeros(3, dtype=float)  # desired acceleration (m/s^2)
        self.angle_ref = np.zeros(3, dtype=float)  # desired angles [phi, theta, psi] (rad)
        self.ang_vel_ref = np.zeros(3, dtype=float)  # desired angular velocity (rad/s)
        self.ang_acc_ref = np.zeros(3, dtype=float)  # desired angular acceleration (rad/s^2)

        # Time parameters
        self.time = 0
        self.dt = dt
       
        # Environment variables
        self.gravity = gravity  # acceleration due to gravity (m/s^2)
        self.density = 1.071778  # air density (kg/m^3)

        # Vehicle constants
        self.num_motors = num_motors
        self.mass = mass  # kg
        self.Ixx = 1.0  # mass-moment of inertia about x-axis (kg-m^2)
        self.Iyy = 1.0  # mass-moment of inertia about y-axis (kg-m^2)
        self.Izz = 1.0  # mass-moment of inertia about z-axis (kg-m^2)
        self.L = 0.175  # length from body center to prop center (m)
        
        # Motor parameters
        self.thrust = mass * gravity  # initial total thrust (N)
        self.speeds = np.ones(num_motors, dtype=float) * 565.9  # initial speeds (RPM)
        self.motor_directions = np.array([1, -1, -1, 1])  # CCW (1) or CW (-1) rotation
        self.tau = np.zeros(3, dtype=float)  # body torque (N-m)

        # Limits
        self.maxT = 16.5  # max thrust from any single motor (N)
        self.minT = 0.5  # min thrust from any single motor (N)
        self.max_angle = math.pi/12  # max angle allowed at any time step (rad)

        # Inertia matrix and gravity vector
        self.I = np.array([[self.Ixx, 0, 0], [0, self.Iyy, 0], [0, 0, self.Izz]], dtype=float)
        self.g = np.array([0, 0, -gravity], dtype=float)

        # VLM parameters
        self.vlm_solver = vlm_solver
        self.quad_propeller_mesh = quad_propeller_mesh
        self.omega_dict = {  # Motor speed initialization
            'Propeller_1': np.array([0, 0, self.speeds[0] * self.motor_directions[0]], dtype=float),
            'Propeller_2': np.array([0, 0, self.speeds[1] * self.motor_directions[1]], dtype=float),
            'Propeller_3': np.array([0, 0, self.speeds[2] * self.motor_directions[2]], dtype=float),
            'Propeller_4': np.array([0, 0, self.speeds[3] * self.motor_directions[3]], dtype=float)
        }
        self.wind_field = None  # Wind field (will be set later)
        self.forces_and_moments = None  # Will store VLM calculated forces and moments
        self.time_step = 0  # Current VLM time step
        
        self.done = False

    def calc_pos_error(self, pos):
        ''' Returns the error between actual position and reference position'''
        pos_error = self.pos_ref - pos
        return pos_error
        
    def calc_vel_error(self, vel):
        ''' Returns the error between actual velocity and reference velocity'''
        vel_error = self.vel_ref - vel
        return vel_error

    def calc_ang_error(self, angle):
        ''' Returns the error between actual angle and reference angle'''
        angle_error = self.angle_ref - angle
        return angle_error

    def calc_ang_vel_error(self, ang_vel):
        ''' Returns the error between angular velocity and reference angular velocity'''
        ang_vel_error = self.ang_vel_ref - ang_vel
        return ang_vel_error

    def body2inertial_rotation(self):
        ''' 
        Euler rotations from body-frame to global inertial frame
        angle 0 = roll (x-axis, phi)
        angle 1 = pitch (y-axis, theta)
        angle 2 = yaw (z-axis, psi)
        '''
        c1 = np.cos(self.angle[0]) 
        s1 = np.sin(self.angle[0])
        c2 = np.cos(self.angle[1])
        s2 = np.sin(self.angle[1])
        c3 = np.cos(self.angle[2])
        s3 = np.sin(self.angle[2])

        R = np.array([
            [c2*c3, c3*s1*s2 - c1*s3, s1*s3 + c1*s2*c3],
            [c2*s3, c1*c3 + s1*s2*s3, c1*s3*s2 - c3*s1],
            [-s2, c2*s1, c1*c2]
        ])
    
        return R

    def inertial2body_rotation(self):
        ''' 
        Euler rotations from inertial to body frame
        (Transpose of body-to-internal rotation)
        '''
        R = np.transpose(self.body2inertial_rotation())
        return R

    def thetadot2omega(self):
        '''rotate body angular velocity (Euler_dot) to inertial angular velocity (omega) '''
        R = np.array([
            [1, 0, -np.sin(self.angle[1])],
            [0, np.cos(self.angle[0]), np.cos(self.angle[1])*np.sin(self.angle[0])],
            [0, -np.sin(self.angle[0]), np.cos(self.angle[1])*np.cos(self.angle[0])]
        ])

        omega = np.matmul(R, self.ang_vel)
        return omega

    def omegadot2Edot(self, omega_dot):
        '''rotate inertial angular velocity (omega) to body angular velocity (Euler_dot) '''
        t1 = np.tan(self.angle[1])
        c1 = np.cos(self.angle[0])
        s1 = np.sin(self.angle[0])
        c2 = np.cos(self.angle[1])

        R = np.array([
            [1, s1*t1, c1*t1],
            [0, c1, -s1],
            [0, s1/c2, c1/c2]
        ])

        E_dot = np.matmul(R, omega_dot)
        self.ang_acc = E_dot

    def find_omegadot(self, omega):
        ''' Find the angular acceleration in the inertial frame in rad/s '''
        omega_dot = np.linalg.inv(self.I).dot(self.tau - np.cross(omega, np.matmul(self.I, omega)))
        return omega_dot

    def update_motor_speeds(self, speeds):
        """Update motor speeds and omega dictionary for VLM calculations"""
        self.speeds = speeds
        self.omega_dict = {
            'Propeller_1': np.array([0, 0, self.speeds[0] * self.motor_directions[0]], dtype=float),
            'Propeller_2': np.array([0, 0, self.speeds[1] * self.motor_directions[1]], dtype=float),
            'Propeller_3': np.array([0, 0, self.speeds[2] * self.motor_directions[2]], dtype=float),
            'Propeller_4': np.array([0, 0, self.speeds[3] * self.motor_directions[3]], dtype=float)
        }

    def calculate_vlm_forces_moments(self, inner_steps=10):
        """
        Calculate forces and moments using the VLM model with multiple azimuth positions
        
        Args:
            inner_steps: Number of inner steps for different propeller azimuth positions
            
        Returns:
            Dictionary with average forces and moments
        """
        inner_forces = []
        inner_moments = []
        
        # Get velocity in body frame for VLM calculation
        velocity_body = self.inertial2body_rotation().dot(self.vel)
        
        # Calculate inner time step size
        inner_dt = self.dt / inner_steps
        
        # Current time step to start from
        current_time_step = self.time_step
        
        # Simulate multiple azimuth positions
        for i in range(inner_steps):
            # Increment time step for this azimuth position
            current_time_step += 1
            
            # Calculate forces and moments for this azimuth position
            forces_and_moments = self.vlm_solver.calculate_total_forces_and_moments(
                propeller_mesh=self.quad_propeller_mesh,
                dt=inner_dt,
                rho=self.density,
                time_step=current_time_step,
                body_velocity=velocity_body,
                omega=self.omega_dict,
                wind_field=self.wind_field,
                com_position=self.pos,
                roll=self.angle[0],
                pitch=self.angle[1],
                yaw=self.angle[2]
            )
            
            # Store results if valid
            if 'total_force' in forces_and_moments and 'total_moment' in forces_and_moments:
                inner_forces.append(forces_and_moments['total_force'])
                inner_moments.append(forces_and_moments['total_moment'])
        
        # Calculate average forces and moments
        if inner_forces and inner_moments:
            avg_force = np.mean(inner_forces, axis=0)
            avg_moment = np.mean(inner_moments, axis=0)
            
            # Update quadcopter with average values
            self.forces_and_moments = {
                'total_force': avg_force,
                'total_moment': avg_moment
            }
            
            # Extract thrust and moments
            self.thrust = np.linalg.norm(avg_force)
            self.tau = avg_moment
            
            # Update time step
            self.time_step = current_time_step
            
            return self.forces_and_moments
        
        return None

    def find_lin_acc(self):
        ''' Find linear acceleration in m/s^2 using forces from VLM'''
        R_B2I = self.body2inertial_rotation()
        
        # Transform forces from body to inertial frame
        if self.forces_and_moments and 'total_force' in self.forces_and_moments:
            force_body = self.forces_and_moments['total_force']
            force_inertial = np.matmul(R_B2I, force_body)
        else:
            # Use simple model if VLM forces not available
            thrust_body = np.array([0, 0, self.thrust], dtype=float)
            force_inertial = np.matmul(R_B2I, thrust_body)
        
        # Add gravity
        weight = self.mass * self.g
        
        # Calculate acceleration
        acc_inertial = (force_inertial + weight) / self.mass
        self.lin_acc = acc_inertial
        
        return self.lin_acc

    def des2speeds(self, thrust_des, tau_des):
        ''' Find motor speeds needed to achieve a desired thrust and torque '''
        # Calculate base speed for desired thrust
        n = self.num_motors
        kt = 1e-7  # Thrust coefficient (N/rpm²)
        b = 1e-9   # Torque coefficient (N·m/rpm²)
        
        base_speed = np.sqrt(thrust_des / (n * kt))
        
        # Calculate speed differentials for desired torques
        dx = tau_des[0] / (2 * self.L * kt)  # Roll torque differential
        dy = tau_des[1] / (2 * self.L * kt)  # Pitch torque differential
        dz = tau_des[2] / (4 * b)            # Yaw torque differential
        
        # Motor configuration for quadcopter (counterclockwise numbering)
        # 1: Front-Left (CCW), 2: Front-Right (CW), 3: Back-Right (CW), 4: Back-Left (CCW)
        motor_speeds = np.array([
            base_speed - dy - dz,  # Front left
            base_speed - dx + dz,  # Front right
            base_speed + dy - dz,  # Back right
            base_speed + dx + dz   # Back left
        ], dtype=float)
        
        # Enforce min/max limits
        max_speed = np.sqrt(self.maxT / kt)
        min_speed = np.sqrt(self.minT / kt)
        motor_speeds = np.clip(motor_speeds, min_speed, max_speed)
        
        # Update speeds
        self.update_motor_speeds(motor_speeds)
        
        return motor_speeds

    def update_dynamics(self):
        """Update the quadcopter dynamics based on current forces and moments"""
        # Calculate linear acceleration
        self.find_lin_acc()
        
        # Calculate angular acceleration
        omega = self.thetadot2omega()
        omega_dot = self.find_omegadot(omega)
        self.omegadot2Edot(omega_dot)
        
        return self.lin_acc, self.ang_acc

    def update_state(self, dt):
        """Update the quadcopter state for the given time step"""
        # Update state variables
        self.ang_vel += dt * self.ang_acc
        self.angle += dt * self.ang_vel
        self.vel += dt * self.lin_acc
        self.pos += dt * self.vel
        self.time += dt


class PID_Controller:
    def __init__(self, Kp, Kd, Ki, Ki_sat, dt):
        self.Kp = np.array(Kp, dtype=float)
        self.Kd = np.array(Kd, dtype=float)
        self.Ki = np.array(Ki, dtype=float)
        self.Ki_sat = np.array(Ki_sat, dtype=float)
        self.e_int = np.zeros(3, dtype=float)
        self.dt = dt
        
    def control_update(self, e, e_dot):
        """PID control update"""
        # Integrate error
        self.e_int += e * self.dt
        
        # Anti-windup - limit integration
        for i in range(3):
            if self.e_int[i] > self.Ki_sat[i]:
                self.e_int[i] = self.Ki_sat[i]
            elif self.e_int[i] < -self.Ki_sat[i]:
                self.e_int[i] = -self.Ki_sat[i]
        
        # Calculate control signal
        u = np.multiply(self.Kp, e) + np.multiply(self.Kd, e_dot) + np.multiply(self.Ki, self.e_int)
        
        return u


def run_integrated_simulation(sim_time=30, dt=0.1, inner_steps=10):
    """
    Run the integrated quadcopter VLM simulation
    
    Args:
        sim_time: Simulation time in seconds
        dt: Time step in seconds
        inner_steps: Number of inner steps for propeller azimuth sampling
    
    Returns:
        Dictionary with simulation results
    """
    print("Initializing VLM components...")
    
    # Initialize propeller geometry and mesh
    propeller_geometry = PropellerGeometry(
        airfoil_distribution_file='DJI9443_airfoils.csv',
        chorddist_file='DJI9443_chorddist.csv',
        pitchdist_file='DJI9443_pitchdist.csv',
        sweepdist_file='DJI9443_sweepdist.csv',
        heightdist_file='DJI9443_heightdist.csv',
        R_tip=0.11938,
        R_hub=0.00624,
        num_blades=2
    )
    
    propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=0.175, com=(0, 0, 0))
    quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
    
    # Initialize the UVLM solver
    uvlm_solver = VLM(quad_propeller_mesh)
    
    # Initial conditions - position at x=325m, y=135m, z=50m
    r_ref = np.array([325., 135., 50.])  # desired position [x, y, z] in meters
    pos = np.array([325., 135., 50.])  # starting location [x, y, z] in meters
    vel = np.array([0., 0., 0.])  # initial velocity [x; y; z] in m/s
    ang = np.array([0., 0., 0.])  # initial Euler angles [phi, theta, psi] in radians
    ang_vel = np.array([0., 0., 0.])  # initial angular rates
    
    # Create quadcopter instance
    quadcopter = QuadcopterVLM(pos, vel, ang, ang_vel, r_ref, dt, uvlm_solver, quad_propeller_mesh)
    
    # Controller gains
    # Position controller gains
    Kp_pos = [.95, .95, 15.]  # proportional [x,y,z]
    Kd_pos = [1.8, 1.8, 15.]  # derivative [x,y,z]
    Ki_pos = [0.2, 0.2, 1.0]  # integral [x,y,z]
    Ki_sat_pos = 1.1 * np.ones(3)  # saturation for integral controller
    
    # Attitude controller gains
    Kp_ang = [6.9, 6.9, 25.]  # proportional [roll,pitch,yaw]
    Kd_ang = [3.7, 3.7, 9.]   # derivative [roll,pitch,yaw]
    Ki_ang = [0.1, 0.1, 0.1]  # integral [roll,pitch,yaw]
    Ki_sat_ang = 0.1 * np.ones(3)  # saturation for integral controller
    
    # Create controllers
    pos_controller = PID_Controller(Kp_pos, Kd_pos, Ki_pos, Ki_sat_pos, dt)
    angle_controller = PID_Controller(Kp_ang, Kd_ang, Ki_ang, Ki_sat_ang, dt)
    
    # Initialize time array
    time_index = np.arange(0, sim_time + dt, dt)
    
    # Initialize results dictionary
    results = {
        'time': time_index,
        'position': [[] for _ in range(3)],
        'velocity': [[] for _ in range(3)],
        'angle': [[] for _ in range(3)],
        'angle_vel': [[] for _ in range(3)],
        'motor_thrust': [[] for _ in range(4)],
        'body_torque': [[] for _ in range(3)],
        'total_error': [],
        'total_thrust': [],
        'wind_velocity': [],
        'wind_file': []  # Track which wind file is used at each time step
    }
    
    # Simulation loop
    print("Starting simulation...")
    for t_idx, t in enumerate(time_index):
        # Print progress
        if t_idx % 10 == 0:
            print(f"Simulation progress: {t/sim_time*100:.1f}%")
        
        # Load the appropriate wind field for this time step
        # Calculate which wind field to use (integer part of time value)
        wind_file_index = int(t) + 1
        wind_file_path = f"bp_50percent_{wind_file_index}.vtk"
        
        try:
            # Load the wind field for this time step
            mesh = pv.read(wind_file_path)
            wind_field = WindField(mesh)
            print(f"Loaded wind field: {wind_file_path}")
            
            # Set the new wind field in quadcopter
            quadcopter.wind_field = wind_field
            
            # Record the wind file used
            results['wind_file'].append(wind_file_path)
        except Exception as e:
            print(f"Error loading wind field {wind_file_path}: {e}")
            print("Using previously loaded wind field or None")
            results['wind_file'].append("None")
            # If no wind field is loaded yet, set to None
            if not hasattr(quadcopter, 'wind_field') or quadcopter.wind_field is None:
                quadcopter.wind_field = None
        
        # ==============================================
        # STEP 1: Calculate VLM forces and moments
        # ==============================================
        # Run inner steps to calculate forces at different azimuth positions
        forces_and_moments = quadcopter.calculate_vlm_forces_moments(inner_steps)
        
        # ==============================================
        # STEP 2: Update quadcopter dynamics with forces
        # ==============================================
        lin_acc, ang_acc = quadcopter.update_dynamics()
        
        # ==============================================
        # STEP 3: Controllers react to update motor speeds
        # ==============================================
        # Position controller
        pos_error = quadcopter.calc_pos_error(quadcopter.pos)
        vel_error = quadcopter.calc_vel_error(quadcopter.vel)
        des_acc = pos_controller.control_update(pos_error, vel_error)
        
        # Modify z acceleration to include thrust required to hover
        des_acc[2] = (quadcopter.gravity + des_acc[2]) / (
            math.cos(quadcopter.angle[0]) * math.cos(quadcopter.angle[1]))
        
        # Calculate thrust needed
        thrust_needed = quadcopter.mass * des_acc[2]
        
        # Calculate desired angles from acceleration
        mag_acc = np.linalg.norm(des_acc)
        if mag_acc == 0:
            mag_acc = 1
        
        ang_des = [
            math.asin(-des_acc[1] / mag_acc / math.cos(quadcopter.angle[1])),
            math.asin(des_acc[0] / mag_acc),
            0  # desired yaw is zero
        ]
        
        # Limit maximum angle
        mag_angle_des = np.linalg.norm(ang_des)
        if mag_angle_des > quadcopter.max_angle:
            ang_des = (np.array(ang_des) / mag_angle_des) * quadcopter.max_angle
        
        # Attitude controller
        quadcopter.angle_ref = ang_des
        ang_error = quadcopter.calc_ang_error(quadcopter.angle)
        ang_vel_error = quadcopter.calc_ang_vel_error(quadcopter.ang_vel)
        tau_needed = angle_controller.control_update(ang_error, ang_vel_error)
        
        # Calculate motor speeds needed
        motor_speeds = quadcopter.des2speeds(thrust_needed, tau_needed)
        
        # ==============================================
        # STEP 4: Update quadcopter state
        # ==============================================
        quadcopter.update_state(dt)
        
        # ==============================================
        # STEP 5: Record results for plotting
        # ==============================================
        # Record positions, angles, velocities
        for i in range(3):
            results['position'][i].append(quadcopter.pos[i])
            results['velocity'][i].append(quadcopter.vel[i])
            results['angle'][i].append(np.rad2deg(quadcopter.angle[i]))
            results['angle_vel'][i].append(np.rad2deg(quadcopter.ang_vel[i]))
            results['body_torque'][i].append(quadcopter.tau[i])
        
        # Record motor thrusts
        kt = 1e-7  # Thrust coefficient
        for i in range(4):
            thrust = quadcopter.speeds[i] * kt
            results['motor_thrust'][i].append(thrust)
        
        # Record total thrust
        results['total_thrust'].append(quadcopter.thrust)
        
        # Record wind velocity at quadcopter position
        if quadcopter.wind_field:
            try:
                wind_vel = quadcopter.wind_field.get_velocity_at_point(quadcopter.pos)
                results['wind_velocity'].append(wind_vel)
            except:
                results['wind_velocity'].append(np.zeros(3))
        else:
            results['wind_velocity'].append(np.zeros(3))
        
        # Positional error
        r_error = quadcopter.pos_ref - quadcopter.pos
        results['total_error'].append(np.linalg.norm(r_error))
    
    print("Simulation complete!")
    return results


def plot_simulation_results(results):
    """Plot the basic simulation results"""
    time_index = results['time']
    position = results['position']
    angle = results['angle']
    
    # Try to import enhanced plotting functions
    try:
        from enhanced_plotting import plot_enhanced_results, plot_wind_analysis
        
        # Use enhanced plotting if available
        plot_enhanced_results(results)
        plot_wind_analysis(results)
        return
    except ImportError:
        print("Enhanced plotting module not found, using basic plots")
    
    # Basic 3D flight path plot
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(position[0], position[1], position[2], linewidth=2)
    ax.set_xlabel('X Position (m)')
    ax.set_ylabel('Y Position (m)')
    ax.set_zlabel('Altitude (m)')
    ax.set_title('Quadcopter Flight Path')
    
    # Add starting position marker
    ax.scatter(position[0][0], position[1][0], position[2][0], 
               color='r', s=100, label='Starting Position')
    
    # Add wind vectors if available
    wind_velocity = results.get('wind_velocity', None)
    if wind_velocity and len(wind_velocity) > 0:
        # Sample wind vectors at regular intervals
        sample_interval = max(1, len(time_index) // 10)
        for i in range(0, len(time_index), sample_interval):
            if i < len(wind_velocity):
                wind_vec = wind_velocity[i]
                scale = 5.0  # Scale factor for visualization
                if np.linalg.norm(wind_vec) > 0:
                    ax.quiver(position[0][i], position[1][i], position[2][i],
                              wind_vec[0] * scale, wind_vec[1] * scale, wind_vec[2] * scale,
                              color='b', alpha=0.5)
    
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Additional plots for aerodynamic analysis
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    # Total thrust vs time
    axs[0, 0].plot(time_index, results['total_thrust'], 'k-')
    axs[0, 0].set_xlabel('Time (s)')
    axs[0, 0].set_ylabel('Thrust (N)')
    axs[0, 0].set_title('Total Thrust vs Time')
    axs[0, 0].grid(True)
    
    # Wind magnitude vs time
    wind_velocity = results.get('wind_velocity', None)
    if wind_velocity and len(wind_velocity) > 0:
        wind_magnitude = [np.linalg.norm(w) for w in wind_velocity]
        axs[0, 1].plot(time_index[:len(wind_magnitude)], wind_magnitude, 'b-')
        axs[0, 1].set_xlabel('Time (s)')
        axs[0, 1].set_ylabel('Wind Speed (m/s)')
        axs[0, 1].set_title('Wind Magnitude vs Time')
        axs[0, 1].grid(True)
    else:
        axs[0, 1].text(0.5, 0.5, 'No Wind Data Available', 
                      horizontalalignment='center', verticalalignment='center',
                      transform=axs[0, 1].transAxes)
    
    # Body torques vs time
    axs[1, 0].plot(time_index, results['body_torque'][0], 'r-', label='Roll')
    axs[1, 0].plot(time_index, results['body_torque'][1], 'g-', label='Pitch')
    axs[1, 0].plot(time_index, results['body_torque'][2], 'b-', label='Yaw')
    axs[1, 0].set_xlabel('Time (s)')
    axs[1, 0].set_ylabel('Torque (N·m)')
    axs[1, 0].set_title('Body Torques vs Time')
    axs[1, 0].legend()
    axs[1, 0].grid(True)
    
    # Angular velocity vs time
    axs[1, 1].plot(time_index, results['angle_vel'][0], 'r-', label='Roll Rate')
    axs[1, 1].plot(time_index, results['angle_vel'][1], 'g-', label='Pitch Rate')
    axs[1, 1].plot(time_index, results['angle_vel'][2], 'b-', label='Yaw Rate')
    axs[1, 1].set_xlabel('Time (s)')
    axs[1, 1].set_ylabel('Angular Velocity (deg/s)')
    axs[1, 1].set_title('Angular Velocity vs Time')
    axs[1, 1].legend()
    axs[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()


def main():
    """Main function to run the simulation"""
    print("Starting integrated quadcopter VLM simulation...")
    
    # Run simulation
    sim_time = 30.0  # seconds
    dt = 0.1         # time step (seconds)
    inner_steps = 10  # number of inner steps per simulation step
    
    results = run_integrated_simulation(sim_time, dt, inner_steps)
    
    # Plot results
    plot_simulation_results(results)
    
    # Save results
    try:
        np.save('quadcopter_vlm_results.npy', results)
        print("Results saved to 'quadcopter_vlm_results.npy'")
    except Exception as e:
        print(f"Error saving results: {e}")
    
    return results


if __name__ == "__main__":
    main()
    
    # Position and angle plots
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    # Position vs time
    axs[0, 0].plot(time_index, position[0], 'r-', label='X')
    axs[0, 0].plot(time_index, position[1], 'g-', label='Y')
    axs[0, 0].plot(time_index, position[2], 'b-', label='Z')
    axs[0, 0].set_xlabel('Time (s)')
    axs[0, 0].set_ylabel('Position (m)')
    axs[0, 0].set_title('Position vs Time')
    axs[0, 0].legend()
    axs[0, 0].grid(True)
    
    # Position error vs time
    axs[0, 1].plot(time_index, results['total_error'], 'm-')
    axs[0, 1].set_xlabel('Time (s)')
    axs[0, 1].set_ylabel('Error (m)')
    axs[0, 1].set_title('Position Error vs Time')
    axs[0, 1].grid(True)
    
    # Angles vs time
    axs[1, 0].plot(time_index, angle[0], 'r-', label='Roll')
    axs[1, 0].plot(time_index, angle[1], 'g-', label='Pitch')
    axs[1, 0].plot(time_index, angle[2], 'b-', label='Yaw')
    axs[1, 0].set_xlabel('Time (s)')
    axs[1, 0].set_ylabel('Angle (deg)')
    axs[1, 0].set_title('Euler Angles vs Time')
    axs[1, 0].legend()
    axs[1, 0].grid(True)
    
    # Motor thrusts vs time
    for i in range(4):
        axs[1, 1].plot(time_index, results['motor_thrust'][i], label=f'Motor {i+1}')
    axs[1, 1].set_xlabel('Time (s)')
    axs[1, 1].set_ylabel('Thrust (N)')
    axs[1, 1].set_title('Motor Thrusts vs Time')
    axs[1, 1].legend()
    axs[1, 1].grid(True)

Starting integrated quadcopter VLM simulation...
Initializing VLM components...
Starting simulation...
Simulation progress: 0.0%


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int32') with casting rule 'same_kind'